In [1]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import mlflow
import mlflow.sklearn
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, median_absolute_error, r2_score

In [2]:
print(os.getcwd())

C:\Users\ifenn\OneDrive\Desktop\Amdari Projects\Jewelry_Price_Optimisation\Notebook


In [5]:
df = pd.read_csv("C:/Users/ifenn/OneDrive/Desktop/Amdari Projects/Jewelry_Price_Optimisation/Data/Jewelry_Dataset.csv")
df.head()

,2018-12-01 11:40:29 UTC,1924719191579951782,1842195256808833386,1,1806829201890738522,jewelry.earring,0,561.51,1515915625207851155,Unnamed: 9,red,gold,diamond
0,2018-12-01 17:38:31 UTC,1924899396621697920,1806829193678291446,1,1.806829e+18,NaN,NaN,212.14,1.515916e+18,NaN,yellow,gold,NaN
1,2018-12-02 13:53:42 UTC,1925511016616034733,1842214461889315556,1,1.806829e+18,jewelry.pendant,1.0,54.66,1.515916e+18,f,white,gold,sapphire
2,2018-12-02 17:44:02 UTC,1925626951238681511,1835566849434059453,1,1.806829e+18,jewelry.pendant,0.0,88.90,1.515916e+18,f,red,gold,diamond
3,2018-12-02 21:30:19 UTC,1925740842841014667,1873936840742928865,1,1.806829e+18,jewelry.necklace,0.0,417.67,1.515916e+18,NaN,red,gold,amethyst
4,2018-12-02 22:09:34 UTC,1925760595336888995,1835566854827934449,1,1.806829e+18,jewelry.earring,1.0,102.27,1.515916e+18,NaN,red,gold,NaN


In [6]:
# Add column names
df.columns = [
    "Order datetime",
    "Order ID",
    "Purchased product ID",
    "Quantity of SKU in the order",
    "Category ID",
    "Category alias",
    "Brand ID",
    "Price in USD",
    "User ID",
    "Product gender",
    "Main Color",
    "Main metal",
    "Main gem",
]

df.head()

,Order datetime,Order ID,Purchased product ID,Quantity of SKU in the order,Category ID,Category alias,Brand ID,Price in USD,User ID,Product gender,Main Color,Main metal,Main gem
0,2018-12-01 17:38:31 UTC,1924899396621697920,1806829193678291446,1,1.806829e+18,NaN,NaN,212.14,1.515916e+18,NaN,yellow,gold,NaN
1,2018-12-02 13:53:42 UTC,1925511016616034733,1842214461889315556,1,1.806829e+18,jewelry.pendant,1.0,54.66,1.515916e+18,f,white,gold,sapphire
2,2018-12-02 17:44:02 UTC,1925626951238681511,1835566849434059453,1,1.806829e+18,jewelry.pendant,0.0,88.90,1.515916e+18,f,red,gold,diamond
3,2018-12-02 21:30:19 UTC,1925740842841014667,1873936840742928865,1,1.806829e+18,jewelry.necklace,0.0,417.67,1.515916e+18,NaN,red,gold,amethyst
4,2018-12-02 22:09:34 UTC,1925760595336888995,1835566854827934449,1,1.806829e+18,jewelry.earring,1.0,102.27,1.515916e+18,NaN,red,gold,NaN


In [7]:
# Display the first few rows and the column names
df.head(), df.columns.tolist()

(            Order datetime             Order ID  Purchased product ID  \
 0  2018-12-01 17:38:31 UTC  1924899396621697920   1806829193678291446   
 1  2018-12-02 13:53:42 UTC  1925511016616034733   1842214461889315556   
 2  2018-12-02 17:44:02 UTC  1925626951238681511   1835566849434059453   
 3  2018-12-02 21:30:19 UTC  1925740842841014667   1873936840742928865   
 4  2018-12-02 22:09:34 UTC  1925760595336888995   1835566854827934449   
 
    Quantity of SKU in the order   Category ID    Category alias  Brand ID  \
 0                             1  1.806829e+18               NaN       NaN   
 1                             1  1.806829e+18   jewelry.pendant       1.0   
 2                             1  1.806829e+18   jewelry.pendant       0.0   
 3                             1  1.806829e+18  jewelry.necklace       0.0   
 4                             1  1.806829e+18   jewelry.earring       1.0   
 
    Price in USD       User ID Product gender Main Color Main metal  Main gem  
 0  

In [8]:
#Data Description
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95910 entries, 0 to 95909
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Order datetime                95910 non-null  object 
 1   Order ID                      95910 non-null  int64  
 2   Purchased product ID          95910 non-null  int64  
 3   Quantity of SKU in the order  95910 non-null  int64  
 4   Category ID                   90558 non-null  float64
 5   Category alias                85977 non-null  object 
 6   Brand ID                      91125 non-null  float64
 7   Price in USD                  90558 non-null  float64
 8   User ID                       90558 non-null  float64
 9   Product gender                47743 non-null  object 
 10  Main Color                    88250 non-null  object 
 11  Main metal                    90448 non-null  object 
 12  Main gem                      61852 non-null  object 
dtypes

In [9]:
# Convert datetime to pandas datetime format
df['Order datetime'] = pd.to_datetime(df['Order datetime'], errors='coerce')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 95910 entries, 0 to 95909
Data columns (total 13 columns):
 #   Column                        Non-Null Count  Dtype              
---  ------                        --------------  -----              
 0   Order datetime                95910 non-null  datetime64[ns, UTC]
 1   Order ID                      95910 non-null  int64              
 2   Purchased product ID          95910 non-null  int64              
 3   Quantity of SKU in the order  95910 non-null  int64              
 4   Category ID                   90558 non-null  float64            
 5   Category alias                85977 non-null  object             
 6   Brand ID                      91125 non-null  float64            
 7   Price in USD                  90558 non-null  float64            
 8   User ID                       90558 non-null  float64            
 9   Product gender                47743 non-null  object             
 10  Main Color                    8825

In [10]:
df.describe(include='all') # Include non-numeric columns too

,Order datetime,Order ID,Purchased product ID,Quantity of SKU in the order,Category ID,Category alias,Brand ID,Price in USD,User ID,Product gender,Main Color,Main metal,Main gem
count,95910,9.591000e+04,9.591000e+04,95910.0,9.055800e+04,85977,9.112500e+04,90558.000000,9.055800e+04,47743,88250,90448,61852
unique,NaN,NaN,NaN,NaN,NaN,218,NaN,NaN,NaN,2,5,3,30
top,NaN,NaN,NaN,NaN,NaN,jewelry.earring,NaN,NaN,NaN,f,red,gold,diamond
freq,NaN,NaN,NaN,NaN,NaN,29051,NaN,NaN,NaN,47379,69510,89081,29609
mean,2021-01-12 18:57:03.605140224+00:00,2.485191e+18,1.815970e+18,1.0,1.805947e+18,NaN,8.891036e+16,362.213017,1.512644e+18,NaN,NaN,NaN,NaN
min,2018-12-01 17:38:31+00:00,1.924899e+18,1.313551e+18,1.0,1.313678e+18,NaN,0.000000e+00,0.990000,1.313554e+18,NaN,NaN,NaN,NaN
25%,2020-08-20 06:48:53.750000128+00:00,2.379732e+18,1.515966e+18,1.0,1.806829e+18,NaN,0.000000e+00,145.620000,1.515916e+18,NaN,NaN,NaN,NaN
50%,2021-03-07 17:25:15+00:00,2.524282e+18,1.956664e+18,1.0,1.806829e+18,NaN,1.000000e+00,258.770000,1.515916e+18,NaN,NaN,NaN,NaN
75%,2021-08-20 09:12:45+00:00,2.644347e+18,1.956664e+18,1.0,1.806829e+18,NaN,1.000000e+00,431.370000,1.515916e+18,NaN,NaN,NaN,NaN
max,2021-12-01 09:59:07+00:00,2.719022e+18,2.541962e+18,1.0,1.806829e+18,NaN,1.550613e+18,34448.600000,1.554297e+18,NaN,NaN,NaN,NaN


In [11]:
#Data Quality Checks
# Missing values per column
df.isnull().sum()

Order datetime                      0
Order ID                            0
Purchased product ID                0
Quantity of SKU in the order        0
Category ID                      5352
Category alias                   9933
Brand ID                         4785
Price in USD                     5352
User ID                          5352
Product gender                  48167
Main Color                       7660
Main metal                       5462
Main gem                        34058
dtype: int64

In [12]:
# Check the columns needed
columns_needed = [
    "Category alias",
    "Brand ID",
    "Price in USD",
    "Product gender",
    "Main Color",
    "Main metal",
    "Main gem"
]

for col in columns_needed:
    print(f"\nTop values for '{col}':")
    print(df[col].value_counts(dropna=False).head(10))


Top values for 'Category alias':
Category alias
jewelry.earring       29051
jewelry.ring          28300
jewelry.pendant       13088
NaN                    9933
jewelry.bracelet       6188
0.00                   4392
jewelry.necklace       2361
jewelry.brooch         1288
jewelry.souvenir        170
electronics.clocks      167
Name: count, dtype: int64

Top values for 'Brand ID':
Brand ID
0.000000e+00    44599
1.000000e+00    32022
2.000000e+00     8966
NaN              4785
1.515916e+18      147
1.515916e+18      104
1.515916e+18       74
4.000000e+00       65
3.000000e+00       58
6.000000e+00       56
Name: count, dtype: int64

Top values for 'Price in USD':
Price in USD
NaN       5352
232.74     945
273.84     855
246.44     814
212.19     814
225.89     811
157.40     776
143.70     739
362.88     730
164.25     709
Name: count, dtype: int64

Top values for 'Product gender':
Product gender
NaN    48167
f      47379
m        364
Name: count, dtype: int64

Top values for 'Main Color

In [13]:
# Unique values per column
df[columns_needed].nunique()

Category alias     218
Brand ID          2537
Price in USD      3166
Product gender       2
Main Color           5
Main metal           3
Main gem            30
dtype: int64

In [14]:
#drop category ID column
df.drop(columns = ["Category ID"], inplace = True)

# drop junk rows from category alias
df = df[~df["Category alias"].isin([np.nan, "nan", "0.00"])]

#convert NaN to Unknown for Brand ID
df["Brand ID"] = df["Brand ID"].fillna("Unknown")

#convert NaN to Unknown for Produce gender
df["Product gender"] = df["Product gender"].fillna("Unknown")

#Replace unknown colour and NaN in main colour with Unkmown
df["Main Color"] = df["Main Color"].replace("unknown-color", "Unknown")
df["Main Color"] = df["Main Color"].fillna("Unknown")

#convert NaN to Unknown for Main Metal
df["Main metal"] = df["Main metal"].fillna("Unknown")

#convert NaN to Unknown for Main gem
df["Main gem"] = df["Main gem"].fillna("Unknown")

#drop rows with NaN for price in usd
df = df.dropna(subset = ["Price in USD"])

#drop user_id column
df.drop(columns = ["User ID"], inplace = True)

In [15]:
# Missing values per column
df.isnull().sum()

Order datetime                  0
Order ID                        0
Purchased product ID            0
Quantity of SKU in the order    0
Category alias                  0
Brand ID                        0
Price in USD                    0
Product gender                  0
Main Color                      0
Main metal                      0
Main gem                        0
dtype: int64

In [16]:
df.nunique()

Order datetime                  63555
Order ID                        63725
Purchased product ID             7920
Quantity of SKU in the order        1
Category alias                      9
Brand ID                            8
Price in USD                     2893
Product gender                      3
Main Color                          4
Main metal                          4
Main gem                           31
dtype: int64